# Introduction to petpy

This notebook introduces the `petpy` package and how to authenticate and interface with the Petfinder API. More information on the Petfinder API itself can be found on their [documentation page](https://www.petfinder.com/developers/api-docs#methods).

# Table of Contents

* [Installing petpy and Obtaining an API key](#installation)
* [Examples using the `petpy` package](#examples)
    - [Pet Methods](#petmethods)
        - [Extracting breeds of animals available in the Petfinder database](#breeds.list)
        - [Returning random Petfinder pet records](#pet.getRandom)
        - [Return a pet record associated with a specific petId](#pet.get)
        - [Finding pet records matching particular search criteria](#pet.find)
    - [Shelter Methods](#sheltermethods)
        - [Finding shelters matching search criteria](#shelter.find)
        - [Returning specific shelter information](#shelter.get)
        - [Extracting pet records from a particular shelter](#shelter.getPets)
        - [Finding shelters that have records matching a particular animal breed](#shelter.listByBreed)

<a id='installation'></a>

## Installation

If not already installed, install `petpy` using `pip`:

``pip install petpy``

Then, import the package.

In [1]:
import petpy

The Petfinder API requires an API key to authenticate access. To receive an API key, register with Petfinder on their developer page: https://www.petfinder.com/developers/api-key

The API key received from Petfinder is then used to authenticate the `Petfinder` class in `petpy`.

The API key is first stored as an environment variable and then loaded using the `os` library. Storing your keys received from APIs and other sensitive information in a secure file or as an environment variable is considered best practice to avoid any potential malicious activity.

In [2]:
import os

with open(os.path.join(os.getcwd(), 'api_key')) as k:
    key = k.read()

In [3]:
pf = petpy.Petfinder(key)

The `pf` variable is the initialized Petfinder class with our given API key. We can now use this instance to interact with and extract data from the Petfinder API.

<a id='#examples'></a>

## Examples using the `petpy` package

<a id='petmethods'></a>

## Pet Methods

The following examples demonstrate some simple usage of using `petpy` to interact with and pull data from the Petfinder database. `petpy` contains methods for coercing the returned API results into a pandas DataFrame for easier data analysis and exporting the results into more common formats such as .csv or Excel. More examples of how to use `petpy` in conjunction with the Python scientific computing stack (Scipy, Numpy, pandas, scikit-learn, etc.) to analyze the results can be found in the later chapters of this introduction.

<a id='breed.list'></a>

### Getting Animal Breeds

Pulling the list of animal breeds from the Petfinder database is straightforward with `petpy`. Let's say we are interested in finding the available breeds of cats:

In [4]:
cats = pf.breed_list('cat')

The default return format is JSON, but can be changed to XML by setting the default parameter `outputformat` to 'xml'.

In [5]:
cats

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'breeds': {'@animal': 'cat',
   'breed': [{'$t': 'Abyssinian'},
    {'$t': 'American Curl'},
    {'$t': 'American Shorthair'},
    {'$t': 'American Wirehair'},
    {'$t': 'Applehead Siamese'},
    {'$t': 'Balinese'},
    {'$t': 'Bengal'},
    {'$t': 'Birman'},
    {'$t': 'Bobtail'},
    {'$t': 'Bombay'},
    {'$t': 'British Shorthair'},
    {'$t': 'Burmese'},
    {'$t': 'Burmilla'},
    {'$t': 'Calico'},
    {'$t': 'Canadian Hairless'},
    {'$t': 'Chartreux'},
    {'$t': 'Chausie'},
    {'$t': 'Chinchilla'},
    {'$t': 'Cornish Rex'},
    {'$t': 'Cymric'},
    {'$t': 'Devon Rex'},
    {'$t': 'Dilute Calico'},
    {'$t': 'Dilute Tortoiseshell'},
    {'$t': 'Domestic Long Hair'},
    {'$t': 'Domestic Long Hair (Black & White)'},
    {'$t': 'Domestic Long Hair (Black)'},
    {'

The `return_df` parameter can also be set to True to coerce the results into a pandas DataFrame.

In [6]:
cats_df = pf.breed_list('cat', return_df=True)
cats_df.head()

,cat breeds
0,Abyssinian
1,American Curl
2,American Shorthair
3,American Wirehair
4,Applehead Siamese


Please note the coercion to a pandas DataFrame removes the metadata returned in the JSON format to make the conversion process more efficient and straightforward.

According to Petfinder's API documentation, the available animals to search are ['barnyard', 'bird', 'cat', 'dog', 'horse', 'reptile', 'smallfurry']. Searching for an animal not available in the Petfinder database will return a JSON object with a message stating 'invalid arguments'.

In [7]:
pf.breed_list('zebra')

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '200'},
    'message': {'$t': 'invalid arguments'}},
   'timestamp': {'$t': '2017-11-23T02:45:17Z'},
   'version': {'$t': '0.1'}}}}

<a id='pet.getRandom'></a>

### Returning random Petfinder pet records

The `petpy` method `pet_getRandom()` provides a wrapper for the Petfinder `pet.getRandom` method. The potential results can be filtered to a subset by the method parameters, otherwise the method can be called simply as:

In [8]:
pf.pet_getRandom()

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2017-11-23T02:45:21Z'},
   'version': {'$t': '0.1'}},
  'petIds': {'id': {'$t': '35781395'}}}}

The default record output contains only the pet record's ID and the call's JSON metadata. If we wish to return a more complete random pet record, we can set the parameter `output` to `basic` (name, age, animal, breed, shelterID) or `full` (complete record with description).

In [9]:
pf.pet_getRandom(output='full')

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2017-11-23T02:45:24Z'},
   'version': {'$t': '0.1'}},
  'pet': {'age': {'$t': 'Adult'},
   'animal': {'$t': 'Dog'},
   'breeds': {'breed': [{'$t': 'Chihuahua'}, {'$t': 'Terrier'}]},
   'contact': {'address1': {},
    'address2': {},
    'city': {'$t': 'Brazil'},
    'email': {'$t': 'luckypupsrescue@gmail.com'},
    'fax': {},
    'phone': {'$t': '(317) 443-2918'},
    'state': {'$t': 'IN'},
    'zip': {'$t': '47834'}},
   'description': {'$t': 'Nacho is an older dog who came to us from another rescue. He is very much a lap dog! he loves to ride in cars and even on the lawn mower! He is housebroken, neutered, up to date on shots, and will be microchipped once adopted. Send us an email at lucky

We can also pull a specified number of pet records from the API by setting the `records` parameter and return the collected results as a pandas DataFrame by setting `return_df` to `True`.

In [10]:
random_pet_df = pf.pet_getRandom(records=5, return_df=True)

In [11]:
random_pet_df

,age,animal,breed0,breed1,breeds.breed,contact.address1,contact.city,contact.email,contact.phone,contact.state,...,photos9,sex,shelterId,shelterPetId,size,status,status0,status1,status2,status3
0,Senior,Cat,Domestic Short Hair,NaN,Domestic Short Hair,4081 Columbia Avenue,Columbia,catmisonly2@yahoo.com,717-684-2285,PA,...,NaN,M,PA696,NaN,S,A,hasShots,altered,housetrained,NaN
0,Adult,Dog,Chihuahua,NaN,Chihuahua,NaN,Romeoville,info@perfectpooches.org,NaN,IL,...,http://photos.petfinder.com/photos/pets/397199...,F,IL753,NaN,S,A,hasShots,altered,NaN,NaN
0,Adult,Cat,Domestic Short Hair (Gray & White),NaN,Domestic Short Hair (Gray & White),NaN,Closter,clawsadopt@yahoo.com,(201) 768-0200,NJ,...,http://photos.petfinder.com/photos/pets/386709...,F,NJ279,Marcia,M,A,hasShots,altered,NaN,NaN
0,Young,Dog,Labrador Retriever,German Shepherd Dog,NaN,PO Box 421,Ben Wheeler,admin@vzchumanesociety.org,903-962-5700,TX,...,http://photos.petfinder.com/photos/pets/317327...,M,TX310,8366590,L,A,specialNeeds,hasShots,altered,housetrained
0,Baby,Cat,Domestic Short Hair,NaN,Domestic Short Hair,P.O. Box 785,Newnan,nchs@nchsrescue.org,NaN,GA,...,NaN,M,GA207,U2017309,S,A,NaN,NaN,NaN,NaN


<a id='pet.get'></a>

### Return a pet record associated with a specific petId

The `pet_get` method can be used to extract a full record from the Petfinder database. We use the pet ID retrieved from the previous call to `pet_getRandom` to illustrate.

In [12]:
pet = pf.pet_get('26417898')

In [13]:
pet

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2017-11-23T02:45:46Z'},
   'version': {'$t': '0.1'}},
  'pet': {'age': {'$t': 'Young'},
   'animal': {'$t': 'Dog'},
   'breeds': {'breed': {'$t': 'American Staffordshire Terrier'}},
   'contact': {'address1': {'$t': '940 Little Britain Rd.'},
    'address2': {},
    'city': {'$t': 'New Windsor'},
    'email': {'$t': 'info@hudsonvalleyspca.org'},
    'fax': {},
    'phone': {'$t': '845-564-6810 '},
    'state': {'$t': 'NY'},
    'zip': {'$t': '12553'}},
   'description': {'$t': 'Zack is a 5 year old m/n very high energy dog.\xa0 He loves people and is very friendly. Because of his high energy level he needs to be a home with older kids and/or an owner who is an experienced dog owner.'},
   'id

The record can also be returned as a `DataFrame`.

In [14]:
pf.pet_get('39801731', return_df=True)

,age,animal,breeds.breed,contact.address1,contact.city,contact.email,contact.phone,contact.state,contact.zip,description,...,photos20,photos21,photos22,photos23,photos24,photos25,photos26,photos27,photos28,photos29
0,Adult,Cat,Domestic Medium Hair (Black & White),54687 County Road 19,Bristol,info@elkharthumanesociety.org,(574) 848-4225,IN,46507,Please visit our website at www.ElkhartHumaneS...,...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...,http://photos.petfinder.com/photos/pets/398017...


The `pets_get()` method accepts a list or tuple and returns the records associated with each pet ID in the passed variable.

In [15]:
petids = random_pet_df['id'].tolist() # get the pet IDs from the previous call by turning the id column into a list

In [16]:
pf.pets_get(petids, return_df=True)

,age,animal,breed0,breed1,breeds.breed,contact.address1,contact.city,contact.email,contact.phone,contact.state,...,photos9,sex,shelterId,shelterPetId,size,status,status0,status1,status2,status3
0,Senior,Cat,Domestic Short Hair,NaN,Domestic Short Hair,4081 Columbia Avenue,Columbia,catmisonly2@yahoo.com,717-684-2285,PA,...,NaN,M,PA696,NaN,S,A,hasShots,altered,housetrained,NaN
0,Adult,Dog,Chihuahua,NaN,Chihuahua,NaN,Romeoville,info@perfectpooches.org,NaN,IL,...,http://photos.petfinder.com/photos/pets/397199...,F,IL753,NaN,S,A,hasShots,altered,NaN,NaN
0,Adult,Cat,Domestic Short Hair (Gray & White),NaN,Domestic Short Hair (Gray & White),NaN,Closter,clawsadopt@yahoo.com,(201) 768-0200,NJ,...,http://photos.petfinder.com/photos/pets/386709...,F,NJ279,Marcia,M,A,hasShots,altered,NaN,NaN
0,Young,Dog,Labrador Retriever,German Shepherd Dog,NaN,PO Box 421,Ben Wheeler,admin@vzchumanesociety.org,903-962-5700,TX,...,http://photos.petfinder.com/photos/pets/317327...,M,TX310,8366590,L,A,specialNeeds,hasShots,altered,housetrained
0,Baby,Cat,Domestic Short Hair,NaN,Domestic Short Hair,P.O. Box 785,Newnan,nchs@nchsrescue.org,NaN,GA,...,NaN,M,GA207,U2017309,S,A,NaN,NaN,NaN,NaN


The `pets_get()` method is essentially a convenience wrapper of `pet_get()`. The same results can be obtained by passing the variable to `pet_get()`.

In [17]:
pf.pet_get(petids, return_df=True)

,age,animal,breed0,breed1,breeds.breed,contact.address1,contact.city,contact.email,contact.phone,contact.state,...,photos9,sex,shelterId,shelterPetId,size,status,status0,status1,status2,status3
0,Senior,Cat,Domestic Short Hair,NaN,Domestic Short Hair,4081 Columbia Avenue,Columbia,catmisonly2@yahoo.com,717-684-2285,PA,...,NaN,M,PA696,NaN,S,A,hasShots,altered,housetrained,NaN
0,Adult,Dog,Chihuahua,NaN,Chihuahua,NaN,Romeoville,info@perfectpooches.org,NaN,IL,...,http://photos.petfinder.com/photos/pets/397199...,F,IL753,NaN,S,A,hasShots,altered,NaN,NaN
0,Adult,Cat,Domestic Short Hair (Gray & White),NaN,Domestic Short Hair (Gray & White),NaN,Closter,clawsadopt@yahoo.com,(201) 768-0200,NJ,...,http://photos.petfinder.com/photos/pets/386709...,F,NJ279,Marcia,M,A,hasShots,altered,NaN,NaN
0,Young,Dog,Labrador Retriever,German Shepherd Dog,NaN,PO Box 421,Ben Wheeler,admin@vzchumanesociety.org,903-962-5700,TX,...,http://photos.petfinder.com/photos/pets/317327...,M,TX310,8366590,L,A,specialNeeds,hasShots,altered,housetrained
0,Baby,Cat,Domestic Short Hair,NaN,Domestic Short Hair,P.O. Box 785,Newnan,nchs@nchsrescue.org,NaN,GA,...,NaN,M,GA207,U2017309,S,A,NaN,NaN,NaN,NaN


<a id='pet.find'></a>

### Finding pet records matching particular search criteria

The `pet.find()` method returns a collection of pet records that match the input search criteria. The available search criteria are listed in the [petpy API documentation](http://petpy.readthedocs.io/en/latest/api.html#pet-methods).

For example, let's say we are interested in finding female cats in Washington state and we want the results returned in a tidy pandas DataFrame.

In [18]:
cats = pf.pet_find(location='WA', animal='cat', sex='F', return_df=True)

In [19]:
cats.head()

,age,animal,contact.address1,contact.city,contact.email,contact.phone,contact.state,contact.zip,description,id,...,photos15,photos16,photos17,photos18,photos19,photos20,photos21,photos22,photos23,photos24
0,Young,Cat,NaN,Auburn,pugetsoundrescue@hotmail.com,NaN,WA,98092,NaN,39898075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Cat,21615 64th Ave S,Kent,adoptapet@kingcounty.gov,206-296-7387,WA,98032,This is my friend Gertie. She is a 10-year-old...,37949653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Senior,Cat,21615 64th Ave S,Kent,adoptapet@kingcounty.gov,206-296-7387,WA,98032,Boo Kitty / Blue Kitty (is that a Russian Blue...,39921493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Senior,Cat,8103 161st Ave NE,Redmond,info@thewhole-cat.com,425-576-5548,WA,98052,Our adoptions program is a relay for 2-6 partn...,38792526,...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...,http://photos.petfinder.com/photos/pets/387925...
0,Adult,Cat,8103 161st Ave NE,Redmond,info@thewhole-cat.com,425-576-5548,WA,98052,Our adoptions program is a relay for 2-6 partn...,38843221,...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...,http://photos.petfinder.com/photos/pets/388432...


The default amount of records returned is 25, which can be changed by setting the `count` parameter. For large queries, it is recommended to set the `pages` parameter with a smaller `count` value. For example, if we wanted to return 1000 results, we could set the `pages` parameter to 10 and the `count` parameter to 100. Please note the Petfinder API places a hard cap of 2,000 results returned per query.

<a id='sheltermethods'></a>

## Shelter Methods

Shelter methods are quite similar to the pet methods explored previously but return information on the animal welfare organizations available in Petfinder's database.

<a id='shelter.find'></a>

### Finding animal welfare organizations in a certain area

The `shelter_find()` method can be used to return shelter records matching the input search criteria. Let's say we want to find 10 shelters listed in the Petfinder database located in Washington State as a pandas DataFrame.

In [20]:
wa_shelters = pf.shelter_find(location='WA', count=10, return_df=True)

In [21]:
wa_shelters

,address1,address2,city,country,email,id,latitude,longitude,name,phone,state,zip
0,NaN,NaN,Auburn,US,pugetsoundrescue@hotmail.com,WA149,47.3191,-122.2682,Puget Sound Rescue,NaN,WA,98092
1,NaN,NaN,Auburn,US,foggycreekcavyrescue@yahoo.com,WA254,47.3191,-122.2682,Foggy Creek Cavy Rescue,NaN,WA,98092
2,21615 64th Ave S,NaN,Kent,US,adoptapet@kingcounty.gov,WA252,47.3747,-122.2775,Regional Animal Services of King County - Fost...,206-296-7387,WA,98032
3,21615 64th Ave S.,NaN,Kent,US,adoptapet@kingcounty.gov,WA63,47.3747,-122.2775,Regional Animal Services of King County,206-296-3936,WA,98032
4,NaN,NaN,Redmond,US,pacosrescue@yahoo.com,WA604,47.3311,-122.3592,Pacos Rescue Network,NaN,WA,98052
5,PO Box 13244,NaN,Des Moines,US,anotherchancecats2007@gmail.com,WA401,47.4043,-122.3105,Another Chance Cat Adoption,(253) 856-1771,WA,98198
6,20613 SE 291ST PL,NaN,Kent,US,PSWC_Adoption@hotmail.com,WA416,47.3718,-122.1474,Puget Sound Working Cats,(206) 819-4261,WA,98042
7,16915 SE 272nd St.,#100-210,Covington,US,smidgetrescue@hotmail.com,WA544,47.3718,-122.1474,Smidget Rescue,206.817.3731,WA,98042
8,NaN,NaN,Covington,US,adoptions.leftbehindk9@gmail.com,WA555,47.3718,-122.1474,Left Behind K-9 Rescue,NaN,WA,98042
9,1102 E. Main Ave,NaN,Puyallup,US,info@sunnyskysshelter.org,WA470,47.2032,-122.2738,Sunny Sky's Animal Rescue,NaN,WA,98372


<a id='shelter.get'></a>

### Returning specific shelter information

The `shelter_get()` method returns the available information in the Petfinder database matching the given shelter ID. Shelter IDs can be found using the `shelter_find()` method used earlier. For example, let's use the method to return the record matching the first shelter ID in the result set obtained in the previous example.

In [22]:
shelter_list = wa_shelters['id'].tolist()

In [23]:
pf.shelter_get(shelter_list[0])

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2017-11-23T02:46:10Z'},
   'version': {'$t': '0.1'}},
  'shelter': {'address1': {},
   'address2': {},
   'city': {'$t': 'Auburn'},
   'country': {'$t': 'US'},
   'email': {'$t': 'pugetsoundrescue@hotmail.com'},
   'fax': {},
   'id': {'$t': 'WA149'},
   'latitude': {'$t': '47.3191'},
   'longitude': {'$t': '-122.2682'},
   'name': {'$t': 'Puget Sound Rescue'},
   'phone': {},
   'state': {'$t': 'WA'},
   'zip': {'$t': '98092'}}}}

The `shelter_get()` method can also accept a list or tuple of shelter IDs. Internally, this calls a convenience wrapper method `shelters_get()`.

In [24]:
pf.shelters_get(shelter_list, return_df=True)

,address1,address2,city,country,email,id,latitude,longitude,name,phone,state,zip
0,NaN,NaN,Auburn,US,pugetsoundrescue@hotmail.com,WA149,47.3191,-122.2682,Puget Sound Rescue,NaN,WA,98092
0,NaN,NaN,Auburn,US,foggycreekcavyrescue@yahoo.com,WA254,47.3191,-122.2682,Foggy Creek Cavy Rescue,NaN,WA,98092
0,21615 64th Ave S,NaN,Kent,US,adoptapet@kingcounty.gov,WA252,47.3747,-122.2775,Regional Animal Services of King County - Fost...,206-296-7387,WA,98032
0,21615 64th Ave S.,NaN,Kent,US,adoptapet@kingcounty.gov,WA63,47.3747,-122.2775,Regional Animal Services of King County,206-296-3936,WA,98032
0,NaN,NaN,Redmond,US,pacosrescue@yahoo.com,WA604,47.3311,-122.3592,Pacos Rescue Network,NaN,WA,98052
0,PO Box 13244,NaN,Des Moines,US,anotherchancecats2007@gmail.com,WA401,47.4043,-122.3105,Another Chance Cat Adoption,(253) 856-1771,WA,98198
0,20613 SE 291ST PL,NaN,Kent,US,PSWC_Adoption@hotmail.com,WA416,47.3718,-122.1474,Puget Sound Working Cats,(206) 819-4261,WA,98042
0,16915 SE 272nd St.,#100-210,Covington,US,smidgetrescue@hotmail.com,WA544,47.3718,-122.1474,Smidget Rescue,206.817.3731,WA,98042
0,NaN,NaN,Covington,US,adoptions.leftbehindk9@gmail.com,WA555,47.3718,-122.1474,Left Behind K-9 Rescue,NaN,WA,98042
0,1102 E. Main Ave,NaN,Puyallup,US,info@sunnyskysshelter.org,WA470,47.2032,-122.2738,Sunny Sky's Animal Rescue,NaN,WA,98372


The result obtained would be the same if one were to use the `shelter_get()` method and passed the same variable.

<a id='shelter.getPets'></a>

### Extracting pet records from a particular shelter

The `shelter.getPets()` method returns the pet records that belong to a particular shelter ID. For example, let's say we want to return the pet records from the first shelter in our list as a DataFrame.

In [25]:
shelter_pets = pf.shelter_getPets(shelter_list[0], return_df=True)
shelter_pets.head() # The default number of returned records is 25, so we only print the first 5 rows for brevity

,age,animal,contact.city,contact.email,contact.state,contact.zip,description,id,lastUpdate,media.photos.photo,...,photos15,photos16,photos17,photos18,photos19,photos20,photos21,photos22,photos23,photos24
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,"Morena is a shepherd/lab mix we are guessing, ...",32080691,2017-09-01T23:26:37Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,Monty is a loving gentleman that likes long wa...,34906970,2017-09-01T23:26:59Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,37760609,2017-11-01T05:01:56Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Young,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,38602092,2017-09-01T21:00:13Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,38627371,2017-11-20T13:54:50Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,http://photos.petfinder.com/photos/pets/386273...,http://photos.petfinder.com/photos/pets/386273...,http://photos.petfinder.com/photos/pets/386273...,http://photos.petfinder.com/photos/pets/386273...,http://photos.petfinder.com/photos/pets/386273...,NaN,NaN,NaN,NaN,NaN


<a id='shelter.listByBreed'></a>

### Finding shelters that have records matching a particular animal breed

The `shelter_listByBreeds()` method allows the user to find shelters that match pet records of a the input animal breed. This method is best used in conjunction with the `breed_list()` method to find the available animal breeds in the Petfinder database.

We already extracted the available cat breeds earlier in the introduction, which we can use to select a cat breed listed in the Petfinder database.

In [26]:
cats_df.head()

,cat breeds
0,Abyssinian
1,American Curl
2,American Shorthair
3,American Wirehair
4,Applehead Siamese


The Abyssinian is a beautiful breed of cat, let's find some shelters that have pet records matching an Abyssinian breed and return it as a DataFrame.

In [27]:
aby = cats_df['cat breeds'].tolist()[0]

In [28]:
aby_shelters = pf.shelter_listByBreed('cat', aby, return_df=True)
aby_shelters.head()

,address1,address2,city,country,email,id,latitude,longitude,name,phone,state,zip
0,5473 Overpass Road,NaN,Santa Barbara,US,SBAdoption@sbcphd.org,CA63,34.4427,-119.8024,Santa Barbara County Animal Services--Santa Ba...,805-681-5285,CA,93111
1,P. O. Box 6356,NaN,Brandon,US,CatcallFL@gmail.com,FL472,27.909,-82.2845,Cat Call Inc.,(813) 324-9320,FL,33511
2,1447 Folly Road,NaN,Charleston,US,adoption@pethelpers.org,SC120,32.7357,-79.9553,Pet Helpers Inc.,843.795.1110,SC,29412
3,6111 Highland Drive,NaN,Jonesboro,US,margaret@neahs.org,AR128,35.8213,-90.6996,NEA Humane Society,(870) 932/5185,AR,72401
4,P.O. Box 1521,NaN,Malvern,US,mataft@netscape.com,AR238,34.1569,-92.9206,Stop Animal Cruelty in Hot Spring County,(501) 276-2385,AR,72104
